# neoPZ Documentation

## Table of contents

1. [Topology](#topology)
2. [Geometry](#geometry)

## 1. Topology <a id="topology"></a>

The topology of all elements implemented on NeoPZ are grouped in pztopology. The classes associated to this namespace are listed in the following:

 
1. TPZPoint
2. TPZLine
3. TPZQuadrilateral
4. TPZTriangle
5. TPZCube
6. TPZPrism
7. TPZPyramid
8. TPZTetrahedron

Each element (e.g. point, line, quadrilateral, triangle, etc) is considered as the union of its sides. Each side is composed of an open set of points ans has inherent dimension and topology.

There is a unique side whose closure is the proper element. By convention this is the highest numbered side.

To each element a parameter space can be associated which is commonly known as the master element space.

### 1.1. Main attributions of Topology class

#### a. Definition of number of corners, sides, dimension and faces.

Side are open sets of points. Within NeoPZ sides are associated with typical finite element topologies: point, line, quadrilateral, triangle, tetrahedra, hexahedra, prism, pyramid.

An element is a closed set of points. Each element can be partioned into its sides (an intersection os its sides is empty and their union is the element).

Zero dimensional sides are called corners. A corner is both open and closed set.

In [1]:
// Make sure to change '/usr/local/' according to the installation in your computer
#pragma cling add_include_path("/usr/local/pzlib/include")
#pragma cling add_include_path("/usr/local/pzlib/include/Topology")
#pragma cling add_include_path("/usr/local/pzlib/include/Matrix")
#pragma cling add_include_path("/usr/local/pzlib/include/Util")
#pragma cling add_include_path("/usr/local/pzlib/include/Common")
#pragma cling add_include_path("/usr/local/pzlib/include/Save")
#pragma cling add_include_path("/usr/local/pzlib/include/Integral")
#pragma cling add_include_path("/usr/local/pzlib/include/LinearSolver")
#pragma cling add_include_path("/usr/local/pzlib/include/PerfUtil")

#pragma cling add_library_path("/usr/local/lib/")

//#pragma cling load("libpz.so")


#include <iostream>

#include "pzvec.h"
//#include "tpzintpoints.h"
//#include "pzquad.h"
#include "tpzquadrilateral.h"

TPZVec<REAL> blob;
blob.resize(3);
blob[0] = 10.;
std::cout << blob[0] << std::endl;

// Link error to be solved:
pztopology::TPZQuadrilateral quad;
std::cout << quad.NCornerNodes << std::endl;
std::cout << quad.NSides << std::endl;
std::cout << quad.Dimension << std::endl;
std::cout << quad.NFaces << std::endl;

SyntaxError: invalid syntax (<ipython-input-1-73ee319dbeb5>, line 1)

#### b. Definition of the dimension of each side and associated corner nodes.

Each side has an associated topological dimension and a given number of corner nodes:

|    Topology   | Dimension | NCornerNodes |
|:-------------:|:---------:|:------------:|
|     Point     |     0     |       1      |
|      Line     |     1     |       2      |
|    Triangle   |     2     |       3      |
| Quadrilateral |     2     |       4      |
|   Tetrahedra  |     3     |       4      |
|   Hexahedra   |     3     |       8      |
|     Prism     |     3     |       6      |
|    Pyramid    |     3     |       5      |

 The graphics of topologies are shown in the Figure 1.

<figure class="image">
  <table><tr><td><img src='images/PointTopo.png'></td><td><img src='images/LineTopo.png'></td></tr></table>
  <table><tr><td><figcaption>(Figure 1A) : The topology of point</figcaption></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td><figcaption>(Figure 1B) : The topology of line</figcaption></td></tr></table>
</figure>

<figure class="image">
  <table><tr><td><img src='images/TriangleTopo.png'></td><td><img src='images/QuadilateralTopo.png'></td></tr></table>
  <table><tr><td><figcaption>(Figure 1C) : The topology of triangle</figcaption></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td><figcaption>(Figure 1D) : The topology of quadrilateral</figcaption></td></tr></table>
</figure>

<figure class="image">
  <table><tr><td><img src='images/TetrahedraTopo.png'></td><td><img src='images/HexahedraTopo.png'></td></tr></table>
  <table><tr><td><figcaption>(Figure 1E) : The topology of tetrahedra</figcaption></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td><figcaption>(Figure 1F) : The topology of hexahedra</figcaption></td></tr></table>
</figure>

The following links show the ProjectPointSideToSide, (Animation):

In [18]:
%%HTML
<video width="500" height="500" controls>
  <source src="sources/triangle.mp4" type="video/mp4">
</video>

In [19]:
%%HTML
<video width="500" height="500" controls>
  <source src="sources/cube.mp4" type="video/mp4">
</video>

##### Table containg orientation for all edges of each topology


<table><tr><td>
    
| Topology | Side | Orientation |
|:--------:|:----:|:-----------:|
| Triangle |      |             |
|          |   3  |     0-1     |
|          |   4  |     1-2     |
|          |   5  |     2-0     | 
|          |      |             |
|          |      |             |
|          |      |             |
|          |      |             |
|          |      |             |
|          |      |             |
|          |      |             |

</td><td>
    
|    Topology   | Side | Orientation |
|:-------------:|:----:|:-----------:|
| Quadrilateral |      |             |
|               |   4  |     0-1     |
|               |   5  |     1-2     |
|               |   6  |     2-3     |
|               |   7  |     3-0     |
|               |      |             |
|               |      |             |
|               |      |             |
|               |      |             |
|               |      |             |

</td><td>    
    
|   Topology  | Side | Orientation |
|:-----------:|:----:|:-----------:|
| Tetrahedron |      |             |
|             |   5  |     0-1     |
|             |   6  |     1-2     |
|             |   7  |     2-0     |
|             |   8  |     0-3     |
|             |   9  |     1-3     |
|             |  10  |     2-3     |

</td></tr></table>

<table><tr><td>
    
|    Topology   |   Side   |   Orientation   |
|:-------------:|:--------:|:---------------:|
|  Hexahedron   |          |                 |
|               |    8     |       0-1       |
|               |    9     |       1-2       |
|               |    10    |       2-3       |
|               |    11    |       3-0       |
|               |    12    |       0-4       |
|               |    13    |       1-5       |
|               |    14    |       2-6       |
|               |    15    |       3-7       |
|               |    16    |       4-5       |
|               |    17    |       5-6       |
|               |    18    |       6-7       |
|               |    19    |       7-4       |

</td><td>
    
|    Topology   |   Side   |   Orientation   |
|:-------------:|:--------:|:---------------:|
|    Pyramid    |          |                 |
|               |    5     |       0-1       |
|               |    6     |       1-2       |
|               |    7     |       2-3       |
|               |    8     |       3-0       |
|               |    9     |       0-4       |
|               |    10    |       1-4       |
|               |    11    |       2-4       |
|               |    12    |       3-4       |
|               |          |                 |
|               |          |                 |
|               |          |                 |
|               |          |                 |
|               |          |                 |
|               |          |                 |
|               |          |                 |
|               |          |                 |
|               |          |                 |
|               |          |                 |

</td><td>    
    
|    Topology   |   Side   |   Orientation   |
|:-------------:|:--------:|:---------------:|
|     Prism     |          |                 |
|               |    6     |       0-1       |
|               |    7     |       1-2       |
|               |    8     |       2-0       |
|               |    9     |       0-3       |
|               |    10    |       1-4       |
|               |    11    |       2-5       |
|               |    12    |       3-4       |
|               |    13    |       4-5       |
|               |    14    |       5-3       |
|               |          |                 |
|               |          |                 |
|               |          |                 |
|               |          |                 |
|               |          |                 |
|               |          |                 |
|               |          |                 |

</td></tr></table>

In [ ]:
# int side = 2;
# int dimside = quad.SideDimension(side);
# std::cout << "The dimension of the side " << side << " is" << dimside << std::endl;
# int nsidenodes = quad.NSideNodes(side);
# std::cout << "The number of associated corner nodes of the side " << side << " is " << nsidenodes << std::endl;

#### c. Definition of the parametric transformation between sides 

Each element is partitioned by its sides. The closure of a side includes a number os sides and forms an element topology. This imples that each point on the boundary of a side belongs to a side of lower dimension.

A point in the parametric domain of a side can be transformed to a point in the parametric domain of the element. The transformation is a homogeneous transformation (i.e. a linear transformation followed by a translation).

These functionalities are implemented by the method *SideToSideTransform*. It is assumed that *sidefrom* belongs to *sideto*. If *sidefrom* corresponds to the volume of the element, the method will return a transformation that is a projection of the interior point to the side. An example of parameteric transformation between sides is shown in the Figure 2.

<figure class="image">
  <img src='images/SideToSideTransform.png'>
  <center>
  <figcaption>(Figure 2) : The transformation between sides.</figcaption>
  </center>
</figure>

In [ ]:
# int sidefrom = 2;
# int sideto = 1;
# TPZTransform<> tr = quad.SideToSideTransform(sidefrom, sideto);

#### d. Creation of integration rules associated to each side

As each side is associated with an element topology, a specific integration rule exists for each side of an element. The integration rule classes associated with each element topology are:

|  **Topology** | **Dimension** | **Class name of integration rule** |
|:-------------:|:-------------:|:----------------------------------:|
|    Abstract   |               |            TPZIntPoints            |
|     Point     |       0       |            TPZInt1Point            |
|      Line     |       1       |              TPZInt1d              |
|    Triangle   |       2       |           TPZIntTriangle           |
| Quadrilateral |       2       |             TPZIntQuad             |
|   Tetrahedra  |       3       |            TPZIntTetra3D           |
|   Hexahedra   |       3       |            TPZIntCube3D            |
|     Prism     |       3       |            TPZIntPrism3D           |
|    Pyramid    |       3       |           TPZIntPyramid3D          |

The method responsible to create the integration rule associated with a side is *CreateSideIntegrationRule*. The input of this method are the *side* and the *order* of the integration rule to be created. The output is a pointer to *TPZIntPoints* which corresponds to the created integration rule.


In [ ]:
# int side = 0;
# int integr_order = 3;
# TPZIntPoints * integr = quad.CreateSideIntegrationRule(side, integr_order)

#### e. Definition of a transformation index associated with a side.

When working with high order hierarchical shape functions the orientation of the side will influence the values of the functions associated with the side. In order to make the functions unique an orientation is associated with each side which depends on an identifier associated with each node. The transformations associated with lines, triangles and quadrilaterals are documented in [NeoPZ shape functions](https://www.sciencedirect.com/science/article/pii/S0045782509000255?via%3Dihub).

The method that computes the transformation index is *int GetTransformId(TPZVec<int64_t> &id)*. The input is a vector with the ids associated with the corner nodes of a side and the output is its the transformation index.

In [2]:
# TPZVec<int64_t> id;
# int transf_id = quad.GetTransformId(id);

#### f. Definition of the permutation index associated with the element side.

When working with H(div) conforming approximations fluxes with continuous normal components are associated with sides of dimension "dim-1" where dim is the dimension of the element. In NeoPZ H(div) compatible flux functions are computed by multiplying a vector field with H1 compatible shape functions. As is documented in [NeoPZ shape functions](https://www.sciencedirect.com/science/article/pii/S0045782509000255?via%3Dihub), H1 shape functions are associated with sides. In order to generate these shape functions in an order compatible between two neighbouring elements, the order is determined by the transformation associated with the side. 
The method that computes the permutation index associated with an element side is *GetSideHDivPermutation(int transformationid, TPZVec &permgather)*. It is implemented for one dimensional, quadrilateral and triangular topologies.



In [3]:
# TPZVec<int> permgather; // output variable
# quad.GetSideHDivPermutation(transf_id, permgather);

#### g. Relationship between sides

This method returns all sides with higher dimension which are included in the closure of a side. The method that shows the higher dimension sides is *HigherDimensionSides(int side, TPZStack &high)*. The input is an integer and the output is a stack.

In [4]:
# TPZStack<int> high;
# quad.HigherDimensionSides(side, high);

#### h. Projection of a point to a side (this function is actually implemented on TPZShape).

When we want to project a point from the interior of the element to a rib in NeoPZ, it is used the method *ProjectPoint2dQuadToRib*. The inputs of this method are the rib index to which the point should be projected and the coordinate of the point at the interior of the element and the output is the coordinate of the point on the rib.
An example of projection of a point to a side is shown in the Figure 3. 

<figure class="image">
  <img src='images/Projected2dToRib.png'>
  <center>
  <figcaption>(Figure 3) : The projection of a point to a side.</figcaption>
  </center>
</figure>

In [5]:
# int rib;
# TPZVec<REAL> in;
# REAL out;
# pzshape::TPZShapeQuad::ProjectPoint2dQuadToRib(rib, in, out);

## 2. Geometry <a id="geometry"></a>

The geometry of all elements types implemented on NeoPZ are grouped in pzgeom. The classes associated to this namespace are listed in the following:
 
1. TPZGeoPoint
2. TPZGeoLinear
3. TPZGeoQuad
4. TPZGeoTriangle
5. TPZGeoCube
6. TPZGeoPrism
7. TPZGeoPyramid
8. TPZGeoTetrahedra

Geometry templates implement the mapping between the master element and actual element. The geometry template also stores the indices of the corner nodes.
The primary attribute of the geometric template is to compute the coordinate as a function of the position in master element coordinate (X) and its gradient (GradX). The standard way of computing X and GradX is by using linear interpolation. As such, the geometric template class implements the linear shape functions associated with its topology.

### 2.1. Fundamental attributes of the Geometric template classes

The geometric template classes are responsible for computing X and GradX for a given point in parameter space. As the only data available in the class are the node indexes (through the super class TPZNodeRep), some data needs to be passed as arguments.
There are two signatures for X and GradX:
* Passing a geometric element pointer as argument. The geometric element is used to access the geometric nodes and their coordinates.
* Passing a matrix with the node coordinates as argument. In this case the computation of X is simply  $x(\xi,\eta)=\sum\phi_i(\xi,\eta)x_i$.

The type of the input/output data is defined as template argument. As such X and GradX can be called with Fad variables.

In [6]:
# don't forget to define the parameters
# TPZGeoEl gel;
# TPZVec<REAL> loc;
# TPZVec<REAL> x;
# TPZVec<REAL> gradx;
# pzgeom::TPZGeoQuad::X(gel, loc, x);
# pzgeom::TPZGeoQuad::GradX(gel, loc, gradx);

In [7]:
# TPZFMatrix<REAL> nodes;
# TPZVec<REAL> loc;
# TPZVec<REAL> x;
# TPZVec<REAL> gradx;
# pzgeom::TPZGeoQuad::X(nodes, loc, x);
# pzgeom::TPZGeoQuad::GradX(nodes, loc, gradx);

### 2.2 Auxiliary methods integrating the geometric template with geometric elements and geometric mesh

#### a. Creating an element along a side

When we want to create a geometric element along a side, in NeoPZ the method is *TPZGeoEl $\ast$CreateBCGeoEl*, in which the geometric template determines the kind of neighbour that needs to be created. 



In [8]:
# don't forget to define the parameters
# TPZGeoEl gel;
# int side = 0;
# int bc = 0;
# TPZGeoEl *ElBC = CreateBCGeoEl(gel, side, bc);

#### b. Inserting an example element

When we want to create an example element based on the topology, in NeoPZ the method is *InsertExampleElement*, in which the geometric template create the element that should be inserted. 
This method is used to create a mesh with a collection of all types of geometric elements available in the library.

In [9]:
# don't forget to define the parameters
# TPZGeoMesh gmesh;
# int matid = 0;
# TPZVec<REAL> lowercorner;
# TPZVec<REAL> size;
# InsertExampleElement(gmesh, matid, lowercorner, size);

#### c. Creating a geometric element

When we want to create an element according to the type of the father element, in NeoPZ the method is *TPZGeoEl $\ast$ CreateGeoElement*.

In [10]:
# don't forget to define the parameters
# TPZGeoMesh gmesh;
# MElementType type = EQuadrilateral;
# TPZVec<int64_t> nodeindexes;
# int matid = 0;
# int64_t index = 0;
# TPZGeoEl *gel = CreateGeoElement(gmesh, type, nodeindexes, matid, index);

#### d. Auxiliary method

The method *IsLinearMapping* declares if the map along the side is linear.
Also, the method *IsGeoBlendEl* indicates if the mapping is originated from a blend element.

## 3. Shape Function <a id="shape"></a>

The origin of the NeoPZ library was an object oriented framework for hp adaptive finite element simulations using continuous H1 approximation spaces.

The shape template classes define shape functions of arbritrary order for all element topologies.

Shape functions are associated with sides and the polynomial order of the shape functions and it can be choosen arbitrarely with each side.

High order shape functions are computed as the product of a bubble function with an orthogonal polynomial.

### 3.1. Choice of family of orthogonal polynomials

The template class *TPZShapeLinear* has a static function pointer *fOrthogonal* that defines the family of orthogonal polynoms that will be used when computing shape functions.

By default, *fOrthogonal* points to the static function *Chebyshev* which implements the orthogonal Chebyshev polynomials.

Other orthogonal polynomials are

 
* Legendre
* Hermite
* Jacobi
* Monomial polynomials $x^n$

### 3.2. Computing shape functions in the NeoPZ environment

Given a point $\xi$ at the interior of an element, the shape functions associated
with a side $s$ of the element topology are computed as:


 
1. Project the point $\xi$ to the point $\xi_s$ associated with the side 

$\xi_{s}=T^{s}(\xi)$;

2. Transform the point according to the transformation id of the side (the transformation id is determined by the $IDs$ of the nodes) 

$\xi_{st}=T^{st}=T^{st}(T^{st}(\xi))$;

3. Compute the internal shape functions for the point 

$\xi_{st}:\phi_{s}(\xi_{st}),  \nabla_{st} \phi_{st}(\xi_{st})$;

4. Multiply the shape functions with the bubble function associated with the side:

$\phi_{bs}(\xi),  \nabla \phi_{bs}(\xi)$

$\phi_{s}(\xi),  \phi_{s}(\xi_{st})\phi_{bs}(\xi)$

$\nabla_{\xi}\phi_{s}(\xi)=\phi_{bs}(\xi)\cdot (\nabla T^{s})^{T}(\nabla T^{s})^{T}\nabla_{st} \phi(\xi_{st})+\phi_{s}(\xi_{st})\cdot\nabla \phi_{bs}(\xi)$


The transformations $T^{st}$ and $T^s$ are homogeneous and have therefore constant
Jacobian. The functions $\phi_{bs}(\xi)$ are bubble functions associated with each side
and are computed in the method ShapeGenerating. The projection $T^s$ is unique
for each side. The transformation $T^s$ is a function of the ids of the geometric
nodes associated with the side.

#### 3.2.1. Shape function count

An essential part of computing the shape functions is to know how many shape functions are associated with each side. Two methods return the number of shape functions
* static int *NConnectShapeF(int side, int order)*:
Computes the number of shape functions associated with a side
* static int *NShapeF(TPZVec<int> \& order)*:
    Sum of shape functions for a given order associated with each side. The order vector has the dimension *NSides-NCornerNodes*

#### 3.2.2. Corner shape functions

The shape functions associated with corner nodes are the traditional linear or bilinear shape functions. 
(For the pyramid topology, the linear shape functions are rational functions.)

#### 3.2.3. ShapeGenerating

A bubble function associated with a side is computed as the multiplication of linear combinations of the corner shape functions.

A bubble function associated with a side is nonzero over the side and all higher dimensional sides whose closure contains the side.

A bubble function associated with a side is zero on all lower dimensional sides include in the closure of the side.

The bubble functions are constructed to that they have minimal polynomial order and fullfill the above requirements. 

The method ShapeGenerating which is defined in all shape templates takes as entry the values of the shape functions and derivatives at the corner nodes and returns the values of the bubble functions and their derivatives.

#### 3.2.4. ShapeInternal
ShapeInternal computes the internal functions associated with the topology of a side.

static void ShapeInternal(TPZVec<REAL> &x, int order, TPZFMatrix<REAL> &phi, TPZFMatrix<REAL> &dphi, int quad_transformation_index);
    
Its arguments are the coordinate of the projected point in the master element of the side, the transformation id associated with the side and its output are the values of the shape functions $\phi(\xi_{st})$ and gradient  $(\nabla T^{st})^T \nabla_{st} \phi_{st}(\xi_{st})$